# **IMPORTAÇÃO DAS BIBLIOTECAS UTILIZADAS E CONEXÃO COM O DRIVE**
PARA EXECUTAR CORRETAMENTE ESTE COLAB:
 - Baixe os arquivos das planilhas e do notebook via github (disponível em: github.com/koitech-API);
 - Em um Google Drive de sua escolha, faça o upload das planilhas e do notebook e execute o código abaixo (pode ser necessário alterar o caminho do repositório no código);
 - Após conceder a permissão de acesso, o código reconhecerá as fontes de dados.

 Obs.: "Executar tudo" fará com que todos os arquivos JSON contendo os gráficos sejam acionados para download.

In [4]:
#BIBLIOTECAS UTILIZADAS
#pandas e numpy (para tratamento dos dados contidos nas planilhas do censo)
import pandas as pd
import numpy as np

#plotly (para construção dos gráficos)
import plotly.express as px
from plotly.utils import PlotlyJSONEncoder
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#google.colab (para acesso e download de arquivos)
from google.colab import drive
from google.colab import files

#json (para converter os gráficos baixados em arquivos .json)
import json

#ACESSANDO O GOOGLE DRIVE PARA TER ACESSO ÀS PLANILHAS
#conectando ao drive
drive.mount('/content/drive')

#listando os arquivos que serão utilizados
!ls "/content/drive/MyDrive/populacao_jovem"


Mounted at /content/drive
ocupadas_etário_2010.xlsx	     populacao_residente_idade_2010.xlsx
ocupadas_etário_2022.xlsx	     populacao_residente_idade_2022.xlsx
ocupadas_salarial_2010.xlsx	     populacao_residente_total_2010.csv
ocupadas_salarial_2022.xlsx	     populacao_residente_total_2022.xlsx
_populacao_jovem_2010_2022.ipynb     populacao_residente_total_SP_2010.xlsx
populacao_nivel_instrucao_2010.xlsx  populacao_superior_completo_2010.xlsx
populacao_nivel_instrucao_2022.xlsx  populacao_superior_completo_2022.xlsx


# **POPULAÇÃO JOVEM EMPREGADA (SJC x SP) 2010 - 2022**

ANO DE 2010 (responsável: Guilherme):

In [ ]:
#atribuindo à variável, o conteúdo da tabela
df_ocupadas_2010 = pd.read_excel('/content/drive/MyDrive/populacao_jovem/ocupadas_etário_2010.xlsx', skiprows=4)

#removendo colunas e linhas desnecessárias / sem utilidade
df_ocupadas_2010= df_ocupadas_2010.drop(df_ocupadas_2010.columns[[1]], axis=1)
df_ocupadas_2010 = df_ocupadas_2010.drop(index=[0, 1])

#atribuindo as legendas corretas para o cabeçalho da tabela
df_ocupadas_2010 = df_ocupadas_2010.rename(columns={'Unnamed: 0': 'Municipio'})
df_ocupadas_2010 = df_ocupadas_2010.rename(columns={'Unnamed: 2': 'Idade'})

#atribuindo valores às linhas sem a identificação de munícipio
df_ocupadas_2010['Municipio'] = df_ocupadas_2010['Municipio'].ffill()

#filtrando para serem exibidos os dados apenas das duas cidades SP e SJC
df_sp_ocupadas_2010 = df_ocupadas_2010[df_ocupadas_2010['Municipio'].isin(['São Paulo (SP)'])]
df_sjc_ocupadas_2010 = df_ocupadas_2010[df_ocupadas_2010['Municipio'].isin(['São José dos Campos (SP)'])]

#criando uma função recursiva para análise das razões entre as cidades
def calcular_razoes_2010(df, nome_cidade):
    #cálculando as populações total e jovem empregada
    populacao_ocupada_total = df.iloc[0, 2]
    populacao_jovem_empregada_total = df.iloc[1:5, 2].sum()

    #razões
    razao_jovens_empregados = (populacao_jovem_empregada_total / populacao_ocupada_total) * 100
    razao_jovens_amostra = (populacao_jovem_empregada_total / df.iloc[0, 2:4].sum()) * 100

    #exibição formatada
    print(f'{nome_cidade} amostra jovem ocupada x {nome_cidade} amostra ocupada: {razao_jovens_empregados:.2f}%')
    print(f'{nome_cidade} amostra jovem ocupada x {nome_cidade} amostra total: {razao_jovens_amostra:.2f}%')
    print()  # linha em branco

    #retornando os valores para uso posterior no gráfico
    return {
        "Cidade": nome_cidade,
        "Jovens / Ocupados (%)": razao_jovens_empregados,
        "Jovens / Total (%)": razao_jovens_amostra
    }

#executando as funções
dados_ocupadas_sp_2010 = calcular_razoes_2010(df_sp_ocupadas_2010, 'SP capital')
dados_ocupadas_sjc_2010 = calcular_razoes_2010(df_sjc_ocupadas_2010, 'SJC')

#exibindo os resultados do dataframe
df_resultados_ocupadas_2010 = pd.DataFrame([dados_ocupadas_sp_2010, dados_ocupadas_sjc_2010])

SP capital amostra jovem ocupada x SP capital amostra ocupada: 31.58%
SP capital amostra jovem ocupada x SP capital amostra total: 17.91%

SJC amostra jovem ocupada x SJC amostra ocupada: 33.37%
SJC amostra jovem ocupada x SJC amostra total: 19.10%



ANO DE 2022 (responsável: Lucas; adapdatação e padronização: Guilherme):

In [ ]:
#seção 01
pd.set_option('display.float_format', "{:.2f}".format)
df_ocupadas_2022 = pd.read_excel('/content/drive/MyDrive/populacao_jovem/ocupadas_etário_2022.xlsx')

df_ocupadas_2022.drop(df_ocupadas_2022.columns[[1]], axis=1, inplace=True)
df_ocupadas_2022.drop(index=[0,1,2,3,4,13], axis=1, inplace=True)

#seção 02
df_ocupadas_2022.columns = ["Município", "Idade", "Ocupadas", "Não ocupadas"]
df_ocupadas_2022["Município"] = df_ocupadas_2022["Município"] .ffill()

#seção 03
df_sp_ocupadas_2022 = df_ocupadas_2022[df_ocupadas_2022["Município"].isin(["São Paulo (SP)"])]
df_sjc_ocupadas_2022 = df_ocupadas_2022[df_ocupadas_2022["Município"].isin(['São José dos Campos (SP)'])]

#seção 04
def calcular_razoes_2022(df, nome_cidade):
    populacao_ocupada_total = df.iloc[0, 2]
    populacao_jovem_empregada_total = df.iloc[1:, 2].sum()

    razao_jovens_empregados = (populacao_jovem_empregada_total / populacao_ocupada_total) * 100
    razao_jovens_amostra = (populacao_jovem_empregada_total / df.iloc[0, 2:].sum()) * 100

    print(f'{nome_cidade} amostra jovem ocupada x {nome_cidade} amostra ocupada: {razao_jovens_empregados:.2f}%')
    print(f'{nome_cidade} amostra jovem ocupada x {nome_cidade} amostra total: {razao_jovens_amostra:.2f}%')
    print()

    return {
        "Cidade": nome_cidade,
        "Jovens / Ocupados (%)": razao_jovens_empregados,
        "Jovens / Total (%)": razao_jovens_amostra
    }

dados_ocupadas_sp_2022 = calcular_razoes_2022(df_sp_ocupadas_2022, 'SP capital')
dados_ocupadas_sjc_2022 = calcular_razoes_2022(df_sjc_ocupadas_2022, 'SJC')

df_resultados_ocupadas_2022 = pd.DataFrame([dados_ocupadas_sp_2022, dados_ocupadas_sjc_2022])

SP capital amostra jovem ocupada x SP capital amostra ocupada: 24.30%
SP capital amostra jovem ocupada x SP capital amostra total: 13.38%

SJC amostra jovem ocupada x SJC amostra ocupada: 24.47%
SJC amostra jovem ocupada x SJC amostra total: 13.27%



GRÁFICO - "POPULAÇÃO JOVEM EMPREGADA x POPULAÇÃO TOTAL EMPREGADA 2010 x 2022"

(responsáveis: Guilherme, Lucas; adaptação e padronização: Guilherme)

In [ ]:
#adicionando a coluna de ano
df_2010 = df_resultados_ocupadas_2010.copy()
df_2010["Ano"] = "2010"

df_2022 = df_resultados_ocupadas_2022.copy()
df_2022["Ano"] = "2022"

#combinando os dois dataframes
df_combinado = pd.concat([df_2010, df_2022])

#transformando o gráfico em formato longo
df_melt = df_combinado.melt(
    id_vars=["Cidade", "Ano"],
    var_name="Indicador",
    value_name="Percentual"
)

#criando o gráfico
fig = px.bar(
    df_melt,
    x="Cidade",
    y="Percentual",
    color="Indicador",
    barmode="group",
    facet_col="Ano",
    text="Percentual",
    title="Razões de Jovens Ocupados nas Amostras (SP Capital x SJC, 2010 e 2022)"
)

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    yaxis_title="Percentual (%)",
    xaxis_title="Cidade",
    uniformtext_minsize=5,
    uniformtext_mode='hide',
    bargap=0.4,
)

fig.show()


EXPORTAÇÃO DO GRÁFICO PARA ARQUIVO JSON (A EXECUÇÃO DESTA LINHA É OPCINAL)

In [ ]:
#salva o arquivo JSON
grafico_razao_jovens_ocupados_sjc_sp_2010_2022 = json.dumps(fig, cls=PlotlyJSONEncoder)

with open("grafico_razao_jovens_ocupados_sjc_sp_2010_2022.json", "w") as f:
    f.write(grafico_razao_jovens_ocupados_sjc_sp_2010_2022)

#faz o download
files.download("grafico_razao_jovens_ocupados_sjc_sp_2010_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **DISTRIBIÇÃO SALARIAL EM SALÁRIOS MÍNIMOS (SJC x SP) 2010 - 2022**

ANO DE 2010 (responsável: Guilherme):

In [ ]:
#atribuindo à variável, o conteúdo da tabela
df_salarial_2010 = pd.read_excel('/content/drive/MyDrive/populacao_jovem/ocupadas_salarial_2010.xlsx', skiprows=4)

#atribuindo as legendas corretas para o cabeçalho da tabela
df_salarial_2010 = df_salarial_2010.rename(columns={'Unnamed: 0': 'Municipio'})
df_salarial_2010 = df_salarial_2010.rename(columns={'Unnamed: 1': 'Faixa Salarial'})

#removendo linha desnecessária
df_salarial_2010.drop(index=18, inplace=True)

#atribuindo valores às linhas sem a identificação de munícipio
df_salarial_2010['Municipio'] = df_salarial_2010['Municipio'].ffill()

#removendo a linha total
df_salarial_2010 = df_salarial_2010[df_salarial_2010['Faixa Salarial'] != 'Total'].reset_index(drop=True)

ANO DE 2022 (rsponsável: Lucas; adaptação e padronização: Guilherme)

In [ ]:
#atribuindo à variável, o conteúdo da tabela
df_salarial_2022 = pd.read_excel('/content/drive/MyDrive/populacao_jovem/ocupadas_salarial_2022.xlsx', skiprows=4)

#atribuindo as legendas corretas para o cabeçalho da tabela
df_salarial_2022 = df_salarial_2022.rename(columns={'Unnamed: 0': 'Municipio'})
df_salarial_2022 = df_salarial_2022.rename(columns={'Unnamed: 1': 'Faixa Salarial'})
df_salarial_2022 = df_salarial_2022.rename(columns={'Total': 'Economicamente ativa'})

#removendo linha desnecessária
df_salarial_2022.drop(index=24, inplace=True)

#atribuindo valores às linhas sem a identificação de munícipio
df_salarial_2022['Municipio'] = df_salarial_2022['Municipio'].ffill()

#removendo a linha total
df_salarial_2022 = df_salarial_2022[df_salarial_2022['Faixa Salarial'] != 'Total'].reset_index(drop=True)

#função para auxiliar a somar e agrupar faixas salariais
def combinar_faixas(df, faixas_origem, faixa_destino, municipios):
    #Soma os valores da coluna 'Economicamente ativa' para as faixas salariais especificadas e adiciona uma nova linha com o total por município.

    #criando o filtro para as faixas que serão combinadas
    filtro = df['Faixa Salarial'].isin(faixas_origem)

    novas_linhas = []

    for mun in municipios:
        soma = df.loc[(df['Municipio'] == mun) & filtro, 'Economicamente ativa'].sum()
        novas_linhas.append({
            'Municipio': mun,
            'Faixa Salarial': faixa_destino,
            'Economicamente ativa': soma
        })

    #removendo as faixas antigas e adicionando as novas
    df = df[~filtro].copy()
    df = pd.concat([df, pd.DataFrame(novas_linhas)], ignore_index=True)

    return df

#organizando as faixas com a função
municipios_alvo = ['São Paulo (SP)', 'São José dos Campos (SP)']

# Até 1/2 de salário mínimo
df_salarial_2022 = combinar_faixas(df_salarial_2022, ['Até 1/4 de salário mínimo', 'Mais de 1/4 a 1/2 salário mínimo'], 'Até 1/2 de salário mínimo', municipios_alvo)

# Mais de 2 a 5 salários mínimos
df_salarial_2022 = combinar_faixas(df_salarial_2022, ['Mais de 2 a 3 salários mínimos', 'Mais de 3 a 5 salários mínimos'], 'Mais de 2 a 5 salários mínimos', municipios_alvo)

# Mais de 10 a 20 salários mínimos
df_salarial_2022 = combinar_faixas(df_salarial_2022, ['Mais de 10 a 15 salários mínimos', 'Mais de 15 a 20 salários mínimos'], 'Mais de 10 a 20 salários mínimos', municipios_alvo)

#para organização do dataframe:
faixa_ordem = [
    'Até 1/4 de salário mínimo',
    'Até 1/2 de salário mínimo',
    'Mais de 1/2 a 1 salário mínimo',
    'Mais de 1 a 2 salários mínimos',
    'Mais de 2 a 5 salários mínimos',
    'Mais de 2 a 3 salários mínimos',
    'Mais de 3 a 5 salários mínimos',
    'Mais de 5 a 10 salários mínimos',
    'Mais de 10 a 20 salários mínimos',
    'Mais de 10 a 15 salários mínimos',
    'Mais de 15 a 20 salários mínimos',
    'Mais de 20 salários mínimos',
    'Sem rendimento'
]

#definindo a ordem das categorias salariais e ordenando o DataFrame
df_salarial_2022['Faixa Salarial'] = pd.Categorical(df_salarial_2022['Faixa Salarial'], categories=faixa_ordem, ordered=True)

df_salarial_2022.sort_values(by=['Municipio', 'Faixa Salarial'], inplace=True)
df_salarial_2022.reset_index(drop=True, inplace=True)

GRÁFICO - "População Economicamente Ativa por Faixa Salarial — Comparativo entre SJC e SP (2010 e 2022)"

(responsáveis: Guilherme, Lucas; adaptação e padronização: Guilherme)

In [ ]:
#adicionando coluna "ano" nos dataframes
df_salarial_2010["Ano"] = 2010
df_salarial_2022["Ano"] = 2022

#combinando os dois dataframes
df_completo = pd.concat([df_salarial_2010, df_salarial_2022], ignore_index=True)

#arrumando os valores que foram omitidos
df_completo.loc[0, 'Faixa Salarial'] = 'Até 1/2 de salário mínimo'
df_completo.loc[8, 'Faixa Salarial'] = 'Até 1/2 de salário mínimo'

#garantindo a ordem correta das faixas salariais
ordem_faixas = [
    "Até 1/2 de salário mínimo",
    "Mais de 1/2 a 1 salário mínimo",
    "Mais de 1 a 2 salários mínimos",
    "Mais de 2 a 5 salários mínimos",
    "Mais de 5 a 10 salários mínimos",
    "Mais de 10 a 20 salários mínimos",
    "Mais de 20 salários mínimos",
    "Sem rendimento"
]

df_completo["Faixa Salarial"] = pd.Categorical(df_completo["Faixa Salarial"], categories=ordem_faixas, ordered=True)

#criando uma coluna combinada de cidade + ano para o gráfico
df_completo["Cidade_Ano"] = df_completo["Municipio"] + " - " + df_completo["Ano"].astype(str)

#gráfico
fig = px.bar(
    df_completo,
    x="Faixa Salarial",
    y="Economicamente ativa",
    color="Cidade_Ano",
    barmode="group",
    text="Economicamente ativa",
    title="População Economicamente Ativa por Faixa Salarial — Comparativo entre SJC e SP (2010 e 2022)"
)

fig.update_traces(texttemplate="%{text:.0f}", textposition="outside")
fig.update_layout(
    xaxis_title="Faixa Salarial",
    yaxis_title="Pessoas Economicamente Ativas",
    legend_title="Município e Ano",
    bargap=0.15,
    xaxis_tickangle=-30
)

fig.show()

EXPORTAÇÃO DO GRÁFICO PARA ARQUIVO JSON (A EXECUÇÃO DESTA LINHA É OPCINAL)

In [ ]:
#salva o arquivo JSON
grafico_distribuicao_salarial_sjc_sp_2010_2022 = json.dumps(fig, cls=PlotlyJSONEncoder)

with open("grafico_distribuicao_salarial_sjc_sp_2010_2022.json", "w") as f:
    f.write(grafico_distribuicao_salarial_sjc_sp_2010_2022)

#faz o download
files.download("grafico_distribuicao_salarial_sjc_sp_2010_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **PROPORÇAO DA POPULAÇÃO JOVEM (SJC x SP) 2010 - 2022**

ANO DE 2010 (responsáveis: João, Robert; padronização: Guilherme)

In [46]:
# === 1. Carregar o arquivo Excel ===
df_residente_2010 = pd.read_excel('/content/drive/MyDrive/populacao_jovem/populacao_residente_idade_2010.xlsx', skiprows=4)

# === 2. Limpar e padronizar ===
df_residente_2010 = df_residente_2010.rename(columns={
    df_residente_2010.columns[0]: "Variavel",
    df_residente_2010.columns[1]: "Idade",
    df_residente_2010.columns[2]: "Condicao",
    df_residente_2010.columns[3]: "Total",
    df_residente_2010.columns[4]: "Homens",
    df_residente_2010.columns[5]: "Mulheres"
})

df_residente_2010 = df_residente_2010[df_residente_2010["Idade"].notna()]
df_residente_2010 = df_residente_2010[df_residente_2010["Idade"] != "Idade"]
df_residente_2010 = df_residente_2010.reset_index(drop=True)

# === 3. Extrair e somar para São José dos Campos ===
sjc_data_2010 = df_residente_2010[df_residente_2010["Variavel"].ffill() == "São José dos Campos (SP)"]
sjc_total_2010 = int(sjc_data_2010[sjc_data_2010["Idade"] == "Total"]["Total"].values[0])

sjc_15_29_2010 = sjc_data_2010[sjc_data_2010["Idade"].isin([
    "15 a 17 anos", "18 ou 19 anos", "20 a 24 anos", "25 a 29 anos"
])]
sjc_sum_15_29_2010 = sjc_15_29_2010["Total"].astype(int).sum()

# === 4. Extrair e somar para São Paulo (capital) ===
sp_data_2010 = df_residente_2010[df_residente_2010["Variavel"].ffill() == "São Paulo (SP)"]
sp_total_2010 = int(sp_data_2010[sp_data_2010["Idade"] == "Total"]["Total"].values[0])

sp_15_29_2010 = sp_data_2010[sp_data_2010["Idade"].isin([
    "15 a 17 anos", "18 ou 19 anos", "20 a 24 anos", "25 a 29 anos"
])]
sp_sum_15_29_2010 = sp_15_29_2010["Total"].astype(int).sum()

# === 5. Dados do estado ===
total_estado_sp_2010 = 41262199

# === 6. Calcular percentuais ===
sjc_percent_15_29_2010 = (sjc_sum_15_29_2010 / sjc_total_2010) * 100
sp_percent_15_29_2010 = (sp_sum_15_29_2010 / sp_total_2010) * 100

# (Opcional) Exibir resultados
print("Soma 15–29 (São José dos Campos):", sjc_sum_15_29_2010)
print("Total São José dos Campos:", sjc_total_2010)
print("Soma 15–29 (São Paulo):", sp_sum_15_29_2010)
print("Total São Paulo:", sp_total_2010)
print("População Total Estado SP:", total_estado_sp_2010)
print(f"\nPercentual da faixa 15–29 em São José dos Campos: {sjc_percent_15_29_2010:.2f}%")
print(f"Percentual da faixa 15–29 em São Paulo: {sp_percent_15_29_2010:.2f}%")

Soma 15–29 (São José dos Campos): 168655
Total São José dos Campos: 629921
Soma 15–29 (São Paulo): 2908499
Total São Paulo: 11253503
População Total Estado SP: 41262199

Percentual da faixa 15–29 em São José dos Campos: 26.77%
Percentual da faixa 15–29 em São Paulo: 25.85%


ANO DE 2022 (responsáveis: João, Robert; padronização: Guilherme)

In [47]:
# === 1. Carregar o arquivo Excel ===
df_residente_2022 = pd.read_excel('/content/drive/MyDrive/populacao_jovem/populacao_residente_idade_2022.xlsx', skiprows=4)

# === 2. Limpar e padronizar ===
df_residente_2022 = df_residente_2022.rename(columns={
    df_residente_2022.columns[0]: "Municipio",
    df_residente_2022.columns[1]: "Condicao",
    df_residente_2022.columns[2]: "Total",
    df_residente_2022.columns[3]: "15 a 19 anos",
    df_residente_2022.columns[4]: "20 a 24 anos",
    df_residente_2022.columns[5]: "25 a 29 anos"
})

# Remover linhas vazias ou que não são municípios
df_residente_2022 = df_residente_2022[df_residente_2022["Municipio"].notna()]
df_residente_2022 = df_residente_2022[~df_residente_2022["Municipio"].str.contains("Fonte", na=False)]
df_residente_2022 = df_residente_2022.reset_index(drop=True)

# === 3. Extrair e somar para São José dos Campos ===
sjc_data_2022 = df_residente_2022[df_residente_2022["Municipio"].str.contains("São José dos Campos", case=False, na=False)].iloc[0]
sjc_total_2022 = int(sjc_data_2022["Total"])
sjc_sum_15_29_2022 = int(sjc_data_2022["15 a 19 anos"]) + int(sjc_data_2022["20 a 24 anos"]) + int(sjc_data_2022["25 a 29 anos"])

# === 4. Extrair e somar para São Paulo (capital) ===
sp_data_2022 = df_residente_2022[df_residente_2022["Municipio"].str.contains("São Paulo", case=False, na=False)].iloc[0]
sp_total_2022 = int(sp_data_2022["Total"])
sp_sum_15_29_2022 = int(sp_data_2022["15 a 19 anos"]) + int(sp_data_2022["20 a 24 anos"]) + int(sp_data_2022["25 a 29 anos"])

# === 5. Dados do estado ===
total_estado_sp_2022 = 44749699  # População total estimada do estado em 2022

# === 6. Calcular percentuais ===
sjc_percent_15_29_2022 = (sjc_sum_15_29_2022 / sjc_total_2022) * 100
sp_percent_15_29_2022 = (sp_sum_15_29_2022 / sp_total_2022) * 100

# === 7. Exibir resultados ===
print("Soma 15–29 (São José dos Campos):", sjc_sum_15_29_2022)
print("Total São José dos Campos:", sjc_total_2022)
print("Soma 15–29 (São Paulo):", sp_sum_15_29_2022)
print("Total São Paulo:", sp_total_2022)
print("População Total Estado SP:", total_estado_sp_2022)
print(f"\nPercentual da faixa 15–29 em São José dos Campos: {sjc_percent_15_29_2022:.2f}%")
print(f"Percentual da faixa 15–29 em São Paulo: {sp_percent_15_29_2022:.2f}%")

Soma 15–29 (São José dos Campos): 144730
Total São José dos Campos: 697054
Soma 15–29 (São Paulo): 2448791
Total São Paulo: 11451999
População Total Estado SP: 44749699

Percentual da faixa 15–29 em São José dos Campos: 20.76%
Percentual da faixa 15–29 em São Paulo: 21.38%


GRÁFICO - "População 15–29 anos vs Total por cidade (2010 e 2022)"

(reponsáveis: João, Robert; adaptação e padronização: Guilherme)

In [48]:
#estruturando um dataframe para sjc e sp
df_grafico_residente_2010_2022 = pd.DataFrame({
    "Ano": [2010, 2010, 2010, 2010, 2022, 2022, 2022, 2022],
    "Local": ["SJC", "SJC", "SP", "SP", "SJC", "SJC", "SP", "SP"],
    "Categoria": ["15-29 anos", "Total", "15-29 anos", "Total", "15-29 anos", "Total", "15-29 anos", "Total"],
    "Populacao": [
        sjc_sum_15_29_2010, sjc_total_2010,
        sp_sum_15_29_2010, sp_total_2010,
        sjc_sum_15_29_2022, sjc_total_2022,
        sp_sum_15_29_2022, sp_total_2022
    ]
})

#gráfico
fig = px.bar(
    df_grafico_residente_2010_2022,
    x="Ano",
    y="Populacao",
    color="Categoria",
    text="Populacao",
    barmode="group",
    facet_col="Local",
    title="População 15–29 anos vs Total por cidade (2010 e 2022)",
    color_discrete_map={"15-29 anos": "#636efa", "Total": "#ef553b"}
)

fig.update_traces(texttemplate="%{text}", textposition="outside")
fig.update_layout(
    uniformtext_minsize=10,
    uniformtext_mode="hide",
    yaxis_title="População",
    legend_title="Categoria",
    margin=dict(t=80, b=50)
)

fig.show()

EXPORTAÇÃO DO GRÁFICO PARA ARQUIVO JSON (A EXECUÇÃO DESTA LINHA É OPCINAL)

In [49]:
#salva o arquivo JSON
grafico_populacao_jovem_residente_sjc_sp_2010_2022 = json.dumps(fig, cls=PlotlyJSONEncoder)

with open("grafico_populacao_jovem_residente_sjc_sp_2010_2022.json", "w") as f:
    f.write(grafico_populacao_jovem_residente_sjc_sp_2010_2022)

#faz o download
files.download("grafico_populacao_jovem_residente_sjc_sp_2010_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

GRÁFICO - "Razão da população 15–29 anos em relação ao total por cidade (2010 e 2022)"

(reponsáveis: João, Robert; adaptação e padronização: Guilherme)

In [50]:
#estruturando um dataframe para sjc e sp
df_grafico_razao_residente_2010_2022 = pd.DataFrame({
    "Ano": [2010, 2010, 2022, 2022],
    "Local": ["SJC", "SP", "SJC", "SP"],
    "Percentual_15_29": [
        sjc_percent_15_29_2010, sp_percent_15_29_2010,
        sjc_percent_15_29_2022, sp_percent_15_29_2022
    ]
})

#gráfico
fig = px.bar(
    df_grafico_razao_residente_2010_2022,
    x="Ano",
    y="Percentual_15_29",
    text="Percentual_15_29",
    color="Local",
    barmode="group",
    title="Razão da população 15–29 anos em relação ao total por cidade (2010 e 2022)",
    color_discrete_map={"SJC": "#636efa", "SP": "#ef553b"}
)

fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")
fig.update_layout(
    yaxis_title="Percentual (%)",
    legend_title="Cidade",
    margin=dict(t=80, b=50),
    bargap=0.4
)

fig.show()

EXPORTAÇÃO DO GRÁFICO PARA ARQUIVO JSON (A EXECUÇÃO DESTA LINHA É OPCINAL)


In [51]:
#salva o arquivo JSON
grafico_razao_jovens_residentes_sjc_sp_2010_2022 = json.dumps(fig, cls=PlotlyJSONEncoder)

with open("grafico_razao_jovens_residentes_sjc_sp_2010_2022.json", "w") as f:
    f.write(grafico_razao_jovens_residentes_sjc_sp_2010_2022)

#faz o download
files.download("grafico_razao_jovens_residentes_sjc_sp_2010_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **PIRÂMIDE ETÁRIA (SJC x SP) 2010 - 2022**

ANO DE 2010 (responsável: Lucas; padronização: Guilherme)

In [5]:
#seção 01
df_etario_2010 = pd.read_csv('/content/drive/MyDrive/populacao_jovem/populacao_residente_total_2010.csv', sep=";")
df_etario_2010_sp = pd.read_excel('/content/drive/MyDrive/populacao_jovem/populacao_residente_total_SP_2010.xlsx')

df_etario_homem_2010 = df_etario_2010[["Masculino","Quantidade idade"]]
df_etario_mulher_2010 = df_etario_2010[["Feminino","Quantidade idade.1"]]

#seção 02
def formata_coluna_2010(df, cidade):
  df.drop("Feminino", axis=1, inplace=True)
  df.rename(columns={"Masculino":"Faixa etária"}, inplace=True)

  df["Quantidade"] = df["Quantidade idade"] + df["Quantidade idade.1"]
  df.drop(["Quantidade idade", "Quantidade idade.1"], axis=1, inplace=True)

  df.insert(0, "Município", cidade)

formata_coluna_2010(df_etario_2010, "São José dos Campos (SP)")
formata_coluna_2010(df_etario_2010_sp, "São Paulo (SP)")

df_etario_2010 = pd.concat([df_etario_2010, df_etario_2010_sp])

#seção 03
filtro_0_1 = df_etario_2010['Faixa etária'] == "Menos de 1 ano de idade"
filtro_1_4 = df_etario_2010['Faixa etária'] == "1 a 4 anos de idade"

soma_0_4 = df_etario_2010[filtro_0_1 | filtro_1_4]["Quantidade"].sum()

nova_linha = pd.DataFrame({"Município": "São José dos Campos (SP)", "Faixa etária": "0 a 4 anos", "Quantidade": soma_0_4}, index=[0])
df_etario_2010 = df_etario_2010[~filtro_0_1 & ~filtro_1_4].copy()

df_etario_2010 = pd.concat([df_etario_2010, nova_linha])

#seção 04
filtro_14 = df_etario_2010["Faixa etária"] == "10 a 14 anos de idade"
df_etario_2010.loc[filtro_14, "Faixa etária"] = "10 a 14 anos"
df_etario_2010.reset_index(drop=True,inplace=True)

#seção 05
faixa_etaria = [
    '0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos',
    '20 a 24 anos', '25 a 29 anos', '30 a 34 anos', '35 a 39 anos',
    '40 a 44 anos', '45 a 49 anos', '50 a 54 anos', '55 a 59 anos',
    '60 a 64 anos', '65 a 69 anos', '70 a 74 anos', '75 a 79 anos',
    '80 a 84 anos', '85 a 89 anos', '90 a 94 anos', '95 a 99 anos',
    '100 anos ou mais'
]

df_etario_2010['Faixa etária'] = pd.Categorical(
    df_etario_2010['Faixa etária'],
    categories=faixa_etaria,
    ordered=True
)

#seção 06
df_etario_2010.sort_values(by=['Faixa etária'], inplace=True)
df_etario_2010.reset_index(drop=True, inplace=True)

ANO DE 2022 (responsável: Lucas; padronização: Guilherme)

In [6]:
#seção 01
df_etario_2022 = pd.read_excel('/content/drive/MyDrive/populacao_jovem/populacao_residente_total_2022.xlsx', sheet_name="População residente (Pessoas)", skiprows=2)

df_etario_2022.iloc[2, 0] = "Unidade da Federação e Município"
df_etario_2022.iloc[2, 1] = "Forma de declaração da idade"
df_etario_2022.iloc[2, 2] = "Total_Ambos"
df_etario_2022.iloc[2, 24] = "Total_Homens"
df_etario_2022.iloc[2, 46] = "Total_Mulheres"

formated_headers = df_etario_2022.iloc[2].values.tolist()

df_etario_2022 = df_etario_2022.drop(index=[0, 1, 2, 649])
df_etario_2022.columns = formated_headers

df_etario_2022 = df_etario_2022.reset_index(drop=True)

#seção 02
dic_colunas = {
    "geral": [],
    "homens": [],
    "mulheres": [],
}

colunas_faixa_etária = []

estado_atual = 'geral'
for i, column in enumerate(df_etario_2022.columns[2:], start=2):
    if column.endswith("Homens"):
        estado_atual = 'homens'

    if column.endswith("Mulheres"):
        estado_atual = 'mulheres'

    if 'anos' in column and column not in colunas_faixa_etária:
      colunas_faixa_etária.append(column)

    dic_colunas[estado_atual].append(i)

df_etario_geral_2022 = df_etario_2022.iloc[:, [0,1] + dic_colunas["geral"]]
df_etario_geral_2022.drop(columns=["Forma de declaração da idade", "Total_Ambos"], axis=1, inplace=True)
df_etario_geral_2022.rename(columns={"Unidade da Federação e Município": "Município"}, inplace=True)

#seção 03
df_etario_geral_2022 = df_etario_geral_2022.melt(id_vars="Município", var_name="Faixa etária", value_name="Quantidade")

df_etario_geral_sjc_2022 = df_etario_geral_2022[df_etario_geral_2022["Município"] == "São José dos Campos (SP)"]
df_etario_geral_sjc_2022.reset_index(drop=True, inplace=True)
df_etario_geral_sp_2022 = df_etario_geral_2022[df_etario_geral_2022["Município"] == "São Paulo (SP)"]
df_etario_geral_sp_2022 = df_etario_geral_sp_2022.reset_index(drop=True)

df_etario_2022 = pd.DataFrame()
df_etario_2022 = pd.concat([df_etario_geral_sjc_2022, df_etario_geral_sp_2022], ignore_index=True)

/tmp/ipython-input-1236242200.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_etario_geral_2022.drop(columns=["Forma de declaração da idade", "Total_Ambos"], axis=1, inplace=True)
/tmp/ipython-input-1236242200.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_etario_geral_2022.rename(columns={"Unidade da Federação e Município": "Município"}, inplace=True)


GRÁFICO - "Pirâmide Etária São José dos Campos vs São Paulo - 2010 x 2022"

(reponsável: Lucas; adaptação e padronização: Guilherme)

In [ ]:
#seção 01
df_etario_2022.insert(3, "Ano", 2022)
df_etario_2010.insert(3, "Ano", 2010)

df_etario = pd.concat([df_etario_2022, df_etario_2010])
df_etario.loc[df_etario["Ano"] == 2010, "Quantidade"] *= -1
df_etario["Quantidade_abs"] = df_etario["Quantidade"].abs()

#seção 02
df_sjc = df_etario[df_etario['Município'] == 'São José dos Campos (SP)']
df_sp  = df_etario[df_etario['Município'] == 'São Paulo (SP)']

hovertemplate = (
    "Município: %{customdata[0]}<br>" +
    "Ano: %{customdata[1]}<br>" +
    "Faixa etária: %{customdata[2]}<extra></extra>"
)


fig = make_subplots(
    rows=1, cols=2,
    shared_yaxes=True,
    horizontal_spacing=0.05,
    subplot_titles=("SJC", "SP")
)

fig.add_trace(
    go.Bar(
        y=df_sjc["Faixa etária"],
        x=df_sjc["Quantidade"],
        orientation='h',
        name='São José dos Campos',
        customdata=df_sjc[["Município", "Ano", "Faixa etária", "Quantidade_abs"]],
        hovertemplate = hovertemplate
    ),
    row=1, col=1,
)

fig.add_trace(
    go.Bar(
        y=df_sp["Faixa etária"],
        x=df_sp["Quantidade"],
        orientation='h',
        name='São Paulo',
        customdata=df_sp[["Município", "Ano", "Faixa etária", "Quantidade_abs"]],
        hovertemplate = hovertemplate
    ),
    row=1, col=2
)

fig.update_traces(
    texttemplate='%{customdata[3]}',
    textposition='inside' )

max_val_sp = int(df_sp["Quantidade_abs"].max())
max_val_sjc = int(df_sjc["Quantidade_abs"].max())

tickvals_sp = np.linspace(-max_val_sp, max_val_sp, 10)
tickvals_sjc = np.linspace(-max_val_sjc, max_val_sjc, 10)

ticktext_sp = [str(abs(int(x))) for x in tickvals_sp]
ticktext_sjc = [str(abs(int(x))) for x in tickvals_sjc]

fig.update_xaxes(tickvals=tickvals_sjc, ticktext=ticktext_sjc, row=1, col=1)
fig.update_xaxes(tickvals=tickvals_sp, ticktext=ticktext_sp, row=1, col=2)

fig.update_layout(
    title="Pirâmide Etária São José dos Campos vs São Paulo - 2010 x 2022",
    barmode='overlay',
    template='plotly_white'
)

fig.show()

EXPORTAÇÃO DO GRÁFICO PARA ARQUIVO JSON (A EXECUÇÃO DESTA LINHA É OPCINAL)

In [32]:
#salva o arquivo JSON
grafico_piramide_etaria_sjc_sp_2010_2022 = json.dumps(fig, cls=PlotlyJSONEncoder)

with open("grafico_piramide_etaria_sjc_sp_2010_2022.json", "w") as f:
    f.write(grafico_piramide_etaria_sjc_sp_2010_2022)

#faz o download
files.download("grafico_piramide_etaria_sjc_sp_2010_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **ANÁLISE EDUCACIONAL pt.1: "ENSINO MÉDIO COMPLETO x SUPERIOR COMPLETO" (SJC x SP) 2010 - 2022**

ANO DE 2010 (responsável: Giovana; padronização: Guilherme)

In [ ]:
#seção 01
df = pd.read_excel('/content/drive/MyDrive/populacao_jovem/populacao_nivel_instrucao_2010.xlsx')

#seção 02
def clean_ibge_dataframe(df):
    df_clean = df.copy()
    new_columns = []
    new_columns.append('municipio')
    new_columns.append('cor_raca')
    new_columns.append('grupo_idade')
    new_columns.append('total')
    new_columns.append('medio_completo_superior_incompleto')
    new_columns.append('superior_completo')

    df_clean.columns = new_columns
    df_clean = df_clean.iloc[6:21].copy()

    df_clean.reset_index(drop=True, inplace=True)

    df_clean['municipio'] = df_clean['municipio'].ffill()

    df_clean['cor_raca'] = df_clean['cor_raca'].ffill()

    numeric_cols = ['total', 'medio_completo_superior_incompleto', 'superior_completo']
    for col in numeric_cols:
        df_clean[col] = df_clean[col].replace('-', pd.NA)
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    df_clean['grupo_idade'] = df_clean['grupo_idade'].fillna('Total')

    return df_clean

df_cleaned = clean_ibge_dataframe(df)

#seção 03
df_clean = df.copy()

df_clean = df.iloc[5:].copy()

df_clean.columns = ['municipio', 'cor_raca', 'grupo_idade', 'total', 'medio_completo_superior_incompleto', 'superior_completo']

df_clean = df_clean.reset_index(drop=True)

df_clean = df_clean.dropna(subset=['municipio'])

#seção 04
df_correto = df.iloc[4:].copy()
df_correto.columns = ['municipio', 'cor_raca', 'grupo_idade', 'total', 'medio_completo_superior_incompleto', 'superior_completo']
df_correto = df_correto.reset_index(drop=True)

df_correto['municipio'] = df_correto['municipio'].fillna(method='ffill')
df_correto = df_correto[df_correto['grupo_idade'] != 'Total']
df_correto = df_correto[df_correto['grupo_idade'].notna()]
df_correto = df_correto[df_correto['municipio'] != 'Fonte: IBGE - Censo Demográfico']

df_correto['medio_completo_superior_incompleto'] = df_correto['medio_completo_superior_incompleto'].replace('-', 0)
df_correto['superior_completo'] = df_correto['superior_completo'].replace('-', 0)
df_correto['medio_completo_superior_incompleto'] = pd.to_numeric(df_correto['medio_completo_superior_incompleto'], errors='coerce')
df_correto['superior_completo'] = pd.to_numeric(df_correto['superior_completo'], errors='coerce')

faixas_desejadas = ['18 e 19 anos', '20 a 24 anos', '25 a 29 anos']
df_final = df_correto[df_correto['grupo_idade'].isin(faixas_desejadas)]


dados_organizados = []

for _, row in df_final.iterrows():
    dados_organizados.append({
        'Localidade': row['municipio'],
        'Faixa Etária': row['grupo_idade'],
        'Nível': 'Ensino Médio Completo',
        'Quantidade': row['medio_completo_superior_incompleto']
    })

    dados_organizados.append({
        'Localidade': row['municipio'],
        'Faixa Etária': row['grupo_idade'],
        'Nível': 'Ensino Superior Completo',
        'Quantidade': row['superior_completo']
    })

df_analise_nivel_instrucao_2010 = pd.DataFrame(dados_organizados)

df_medio = df_analise_nivel_instrucao_2010[df_analise_nivel_instrucao_2010['Nível'] == 'Ensino Médio Completo']
df_medio_pivot = df_medio.pivot_table(index='Localidade', columns='Faixa Etária', values='Quantidade')

df_superior = df_analise_nivel_instrucao_2010[df_analise_nivel_instrucao_2010['Nível'] == 'Ensino Superior Completo']
df_superior_pivot = df_superior.pivot_table(index='Localidade', columns='Faixa Etária', values='Quantidade')

/tmp/ipython-input-225896968.py:51: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipython-input-225896968.py:57: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



GRÁFICO - "Análise de Escolaridade: São José dos Campos vs São Paulo  2010"

(responsável: Giovana; adaptação e padronização:: Guilherme)

In [ ]:
#filtrando os dados por nível
df_medio = df_analise_nivel_instrucao_2010[df_analise_nivel_instrucao_2010['Nível'] == 'Ensino Médio Completo']
df_superior = df_analise_nivel_instrucao_2010[df_analise_nivel_instrucao_2010['Nível'] == 'Ensino Superior Completo']

#gráficos
fig = px.bar(
    df_analise_nivel_instrucao_2010,
    x='Localidade',
    y='Quantidade',
    color='Faixa Etária',
    barmode='group',
    facet_col='Nível',
    category_orders={
        'Faixa Etária': ['18 e 19 anos', '20 a 24 anos', '25 a 29 anos'],
        'Localidade': ['São José dos Campos (SP)', 'São Paulo', 'São Paulo (SP)']
    },
    text='Quantidade',
    title='Análise de Escolaridade: São José dos Campos vs São Paulo  2010'
)

fig.update_traces(texttemplate="%{text}", textposition="outside")
fig.update_layout(
    yaxis_title='Quantidade de Pessoas',
    xaxis_title='Localidade',
    legend_title='Faixas Etárias',
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

fig.show()

EXPORTAÇÃO DO GRÁFICO PARA ARQUIVO JSON (A EXECUÇÃO DESTA LINHA É OPCINAL)

In [ ]:
#salva o arquivo JSON
grafico_escolaridade_sjc_sp_2010 = json.dumps(fig, cls=PlotlyJSONEncoder)

with open("grafico_escolaridade_sjc_sp_2010.json", "w") as f:
    f.write(grafico_escolaridade_sjc_sp_2010)

#faz o download
files.download("grafico_escolaridade_sjc_sp_2010.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ANO DE 2022 (responsáveis: Mariana, Rayssa; padronização: Guilherme)

In [ ]:
#seção 01
df = pd.read_excel('/content/drive/MyDrive/populacao_jovem/populacao_nivel_instrucao_2022.xlsx')

#seção 02
ensino_medio = pd.DataFrame({
    'Faixa Etária': ['18 a 19', '20 a 24', '25 a 29'],
    'Estado de SP': [729866, 2088031, 1650113],
    'São Paulo (SP)': [177468, 517313, 389612],
    'São José dos Campos (SP)': [7176, 22181, 26144]
})

ensino_superior = pd.DataFrame({
    'Faixa Etária': ['18 a 19', '20 a 24', '25 a 29'],
    'Estado de SP': [20157, 388404, 871161],
    'São Paulo (SP)': [6492, 129993, 294491],
    'São José dos Campos (SP)': [211, 5968, 15680]
})

#seção 03
ensino_medio = {
    '18 a 19': [729866, 177468, 7176],
    '20 a 24': [2088031, 517313, 22181],
    '25 a 29': [1650113, 389612, 26144]
}

ensino_superior = {
    '18 a 19': [20157, 6492, 211],
    '20 a 24': [388404, 129993, 5968],
    '25 a 29': [871161, 294491, 15680]
}

regioes = ['Estado de SP', 'São Paulo (SP)', 'São José dos Campos (SP)']
faixas = ['18 a 19', '20 a 24', '25 a 29']

dados_organizados = []

for i, regiao in enumerate(regioes):
    for faixa in faixas:
        dados_organizados.append({
            'Localidade': regiao,
            'Faixa Etária': faixa,
            'Nível': 'Ensino Médio Completo',
            'Quantidade': ensino_medio[faixa][i]
        })
        dados_organizados.append({
            'Localidade': regiao,
            'Faixa Etária': faixa,
            'Nível': 'Ensino Superior Completo',
            'Quantidade': ensino_superior[faixa][i]
        })

df_analise_nivel_instrucao_2022 = pd.DataFrame(dados_organizados)

df_medio = df_analise_nivel_instrucao_2022[df_analise_nivel_instrucao_2022['Nível'] == 'Ensino Médio Completo']
df_superior = df_analise_nivel_instrucao_2022[df_analise_nivel_instrucao_2022['Nível'] == 'Ensino Superior Completo']

df_medio_pivot = df_medio.pivot_table(index='Localidade', columns='Faixa Etária', values='Quantidade')
df_superior_pivot = df_superior.pivot_table(index='Localidade', columns='Faixa Etária', values='Quantidade')

GRÁFICO - "Análise de Escolaridade: São José dos Campos vs São Paulo 2022"

(responsáveis: Mariana, Rayssa; adaptação e padronização: Guilherme)

In [ ]:
#organizando as categorias
df_analise_nivel_instrucao_2022['Localidade'] = pd.Categorical(
    df_analise_nivel_instrucao_2022['Localidade'],
    categories=['São José dos Campos (SP)', 'São Paulo (SP)', 'Estado de SP'],
    ordered=True
)

df_analise_nivel_instrucao_2022['Faixa Etária'] = pd.Categorical(
    df_analise_nivel_instrucao_2022['Faixa Etária'],
    categories=['18 a 19', '20 a 24', '25 a 29'],
    ordered=True
)

df_analise_nivel_instrucao_2022['Nível'] = pd.Categorical(
    df_analise_nivel_instrucao_2022['Nível'],
    categories=['Ensino Médio Completo', 'Ensino Superior Completo'],
    ordered=True
)

#gráficos
fig = px.bar(
    df_analise_nivel_instrucao_2022,
    x='Localidade',
    y='Quantidade',
    color='Faixa Etária',
    barmode='group',
    facet_col='Nível',
    category_orders={
        'Faixa Etária': ['18 a 19', '20 a 24', '25 a 29'],
        'Localidade': ['São José dos Campos (SP)', 'Estado de SP', 'São Paulo (SP)']
    },
    text='Quantidade',
    title='Análise de Escolaridade: São José dos Campos vs São Paulo 2022'
)

fig.update_traces(texttemplate="%{text}", textposition="outside")
fig.update_layout(
    yaxis_title='Quantidade de Pessoas',
    xaxis_title='Localidade',
    legend_title='Faixas Etárias',
    uniformtext_minsize=8,
    uniformtext_mode='hide'
)

fig.show()

EXPORTAÇÃO DO GRÁFICO PARA ARQUIVO JSON (A EXECUÇÃO DESTA LINHA É OPCINAL)

In [ ]:
#salva o arquivo JSON
grafico_escolaridade_sjc_sp_2022 = json.dumps(fig, cls=PlotlyJSONEncoder)

with open("grafico_escolaridade_sjc_sp_2022.json", "w") as f:
    f.write(grafico_escolaridade_sjc_sp_2022)

#faz o download
files.download("grafico_escolaridade_sjc_sp_2022.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **ANÁLISE EDUCACIONAL pt.2: "ENSINO SUPERIOR COMPLETO x GRADUAÇÃO EM TI" (SJC x SP) 2010 - 2022**

ANO DE 2010 (responsável: Giovana; padronização: Guilherme)

In [ ]:
df_2010 = pd.read_excel('/content/drive/MyDrive/populacao_jovem/populacao_superior_completo_2010.xlsx')

dados_2010 = [
    ['Estado de São Paulo', 3924436, 329590],
    ['São Paulo (cidade)', 1570884, 118902],
    ['São José dos Campos', 67827, 4252],
]

df_2010 = pd.DataFrame(dados_2010, columns=["Localidade", "Superior Total", "Formação TI"])

df_2010['% TI'] = (df_2010['Formação TI'] / df_2010['Superior Total'] * 100).round(2)
df_2010['% Não-TI'] = 100 - df_2010['% TI']
df_2010['Não-TI'] = df_2010['Superior Total'] - df_2010['Formação TI']

ANO DE 2022 (responsáveis: Mariana, Rayssa; padronização: Guilherme)

In [ ]:
df_2022 = pd.read_excel('/content/drive/MyDrive/populacao_jovem/populacao_superior_completo_2022.xlsx')

dados_2022 = [
    ['Estado de São Paulo', 7478587, 331924],
    ['São Paulo (cidade)', 2512570, 125296],
    ['São José dos Campos', 142053, 6041],
]

df_2022 = pd.DataFrame(dados_2022, columns=["Localidade", "Total Formandos", "Formandos TI"])

# Calculando percentuais e outras métricas
df_2022['% TI'] = (df_2022['Formandos TI'] / df_2022['Total Formandos'] * 100).round(2)
df_2022['% Não-TI'] = 100 - df_2022['% TI']
df_2022['Formandos Outras Áreas'] = df_2022['Total Formandos'] - df_2022['Formandos TI']
df_2022['% do Estado (TI)'] = (df_2022['Formandos TI'] / df_2022.loc[0, 'Formandos TI'] * 100).round(2)
df_2022['% do Estado (Total)'] = (df_2022['Total Formandos'] / df_2022.loc[0, 'Total Formandos'] * 100).round(2)

GRÁFICO - "Formação TI x Total: São José dos Campos vs São Paulo (2010 e 2022)"

(responsáveis: Giovana, Mariana, Rayssa; adaptação e padronização:: Guilherme)

In [ ]:
#atribuindo os dados de 2010
dados_2010 = [
    ['São José dos Campos (SP)', 67827, 4252],
    ['São Paulo', 3924436, 329590],
    ['São Paulo (SP)', 1570884, 118902]
]

df_2010 = pd.DataFrame(dados_2010, columns=["Localidade", "Total Formandos", "Formandos TI"])
df_2010["Ano"] = 2010

#atribuindo os dados de 2022
dados_2022 = [
    ['São José dos Campos (SP)', 142053, 6041],
    ['São Paulo', 7478587, 331924],
    ['São Paulo (SP)', 2512570, 125296]
]

df_2022 = pd.DataFrame(dados_2022, columns=["Localidade", "Total Formandos", "Formandos TI"])
df_2022["Ano"] = 2022

#combinando os dataframes
df_comb = pd.concat([df_2010, df_2022])

#convertendo o dataframe pro formato longo
df_long = df_comb.melt(
    id_vars=["Localidade", "Ano"],
    value_vars=["Total Formandos", "Formandos TI"],
    var_name="Categoria",
    value_name="Quantidade"
)

#gráficos
fig = px.bar(
    df_long,
    x="Ano",
    y="Quantidade",
    color="Categoria",
    text="Quantidade",
    barmode="group",
    facet_col="Localidade",
    title="Formação TI x Total: São José dos Campos vs São Paulo (2010 e 2022)",
    color_discrete_map={"Formandos TI": "#636efa", "Total Formandos": "#ef553b"}
)

fig.update_traces(texttemplate="%{text:,}", textposition="outside")
fig.update_layout(
    uniformtext_minsize=10,
    uniformtext_mode="hide",
    yaxis_title="Quantidade de Formandos",
    legend_title="Categoria",
    margin=dict(t=80, b=50),
    font=dict(size=12)
)

#removendo "Localidade=” do título dos gráficos
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.show()

EXPORTAÇÃO DO GRÁFICO PARA ARQUIVO JSON (A EXECUÇÃO DESTA LINHA É OPCINAL)

In [ ]:
#salva o arquivo JSON
grafico_superior_ti_2010_2022 = json.dumps(fig, cls=PlotlyJSONEncoder)

with open("grafico_superior_ti_2010_2022.json", "w") as f:
    f.write(grafico_superior_ti_2010_2022)

#faz o download
files.download("grafico_superior_ti_2010_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

GRÁFICO - "Proporção de Formandos em TI: São José dos Campos vs São Paulo (2010 e 2022)"

(responsáveis: Giovana, Mariana, Rayssa; adaptação e padronização:: Guilherme)

In [ ]:
#atribuindo os dados de 2010
dados_2010 = [
    ['São José dos Campos (SP)', 67827, 4252],
    ['São Paulo', 3924436, 329590],
    ['São Paulo (SP)', 1570884, 118902]
]

df_2010 = pd.DataFrame(dados_2010, columns=["Localidade", "Total Formandos", "Formandos TI"])
df_2010["Ano"] = 2010
df_2010["% TI"] = (df_2010["Formandos TI"] / df_2010["Total Formandos"] * 100).round(2)

#atribuindo os dados de 2022
dados_2022 = [
    ['São José dos Campos (SP)', 142053, 6041],
    ['São Paulo', 7478587, 331924],
    ['São Paulo (SP)', 2512570, 125296]
]

df_2022 = pd.DataFrame(dados_2022, columns=["Localidade", "Total Formandos", "Formandos TI"])
df_2022["Ano"] = 2022
df_2022["% TI"] = (df_2022["Formandos TI"] / df_2022["Total Formandos"] * 100).round(2)

#combinando os dataframes
df_comb = pd.concat([df_2010, df_2022])

#formatando um dataframe para os gráficos
df_percentual = df_comb[["Localidade", "Ano", "% TI"]].copy()
df_percentual["Categoria"] = "% TI"

#arrumando o ano das tabelas
df_percentual["Ano"] = df_percentual["Ano"].astype(str)

#gráficos
fig = px.bar(
    df_percentual,
    x="Ano",
    y="% TI",
    color="Categoria",
    text="% TI",
    barmode="group",
    facet_col="Localidade",
    title="Proporção de Formandos em TI: São José dos Campos vs São Paulo (2010 e 2022)",
    color_discrete_map={"% TI": "#636efa"}
)

fig.update_traces(texttemplate="%{text}%", textposition="outside")
fig.update_layout(
    uniformtext_minsize=10,
    uniformtext_mode="hide",
    yaxis_title="% de Formandos em TI",
    legend_title="Indicador",
    margin=dict(t=80, b=50),
    font=dict(size=12),
    xaxis_title="Ano"
)

#removendo “Localidade=” dos títulos dos gráficos
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

fig.show()

EXPORTAÇÃO DO GRÁFICO PARA ARQUIVO JSON (A EXECUÇÃO DESTA LINHA É OPCINAL)

In [ ]:
#salva o arquivo JSON
grafico_proporcao_formacao_ti_2010_2022 = json.dumps(fig, cls=PlotlyJSONEncoder)

with open("grafico_proporcao_formacao_ti_2010_2022.json", "w") as f:
    f.write(grafico_proporcao_formacao_ti_2010_2022)

#faz o download
files.download("grafico_proporcao_formacao_ti_2010_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>